# PyCity Schools Analysis

* Overall, charter schools dramatically out-performed district schools on overall passing rates. It is also worth noting, however, that the charter schools at the top with regard to performance were also significantly smaller with regard to student population.


In [112]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Files to Load
school_data_file = "Resources/schools_complete.csv"
student_data_file = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_file)
student_data = pd.read_csv(student_data_file)

# Combine the data into a single dataset
data_combined = pd.merge(student_data, school_data, how="left", on="school_name")
data_combined.head(5)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [2]:
# Capture totals for schools, students, and budget
tot_schools = school_data["school_name"].count()
tot_students = student_data["Student ID"].count()
tot_budget = school_data["budget"].sum()

# Get averages of math and reading scores for the district
avg_math = data_combined["math_score"].mean()
avg_read = data_combined["reading_score"].mean()

# Calculate percent passing Math (score equal to or greater than 70)
math_scores = pd.Series(data_combined["math_score"])
math_count = len([score for score in math_scores if score >= 70])
pct_math = (math_count*100)/tot_students

# Calculate percent passing Reading (score equal to or greater than 70)
read_scores = pd.Series(data_combined["reading_score"])
read_count = len([score for score in read_scores if score >= 70])
pct_read = (read_count*100)/tot_students

# Calculate Overall Passing Rate, defined in instructions as the average of both reading and math averages
overall_score = (avg_math + avg_read) / 2


In [3]:
# Create Data Frame to hold above values
dist_summ = pd.DataFrame({"Total Schools":[tot_schools],"Total Students":[tot_students],"Total Budget":[tot_budget],
                          "Average Math Score":[avg_math],"Average Reading Score":[avg_read],"% Passing Math":[pct_math],
                         "% Passing Reading":[pct_read],"% Overall Passing Rate":[overall_score]})

# Format Total Students and Total Budget columns
dist_summ["Total Students"] = dist_summ["Total Students"].map("{:,}".format)
dist_summ["Total Budget"] = dist_summ["Total Budget"].map("${:,}".format)

# Display DataFrame
dist_summ

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428",78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [109]:
# Create reduced table from data_combined with columns needed
school_summ = data_combined[["school_name","Student ID","reading_score","math_score","type",
                             "size","budget"]]
# school_summ.sample(7)

# Create series for count of students passing math by school
pass_math = school_summ[(school_summ['math_score'] >= 70)]
pass_math = pass_math[["school_name","math_score"]]
pass_math_count = pass_math.groupby('school_name').count()
pass_math_count_ser = pd.Series(pass_math_count['math_score'])
# pass_math_count_ser

# Create series for count of students passing reading by school
pass_read = school_summ[(school_summ['reading_score'] >= 70)]
pass_read = pass_read[["school_name","reading_score"]]
pass_read_count = pass_read.groupby('school_name').count()
pass_read_count_ser = pd.Series(pass_read_count['reading_score'])
# pass_read_count_ser

In [110]:
# Group combined data by schools
grp_school = data_combined.groupby('school_name')

# Calculate metrics per school
sch_type = grp_school['type'].unique()
tot_student_s = grp_school['Student ID'].count()
tot_budget_s = grpSchool['budget'].unique()
stu_budget = tot_budget_s/tot_student_s
avg_math_s = grpSchool['math_score'].mean()
avg_read_s = grpSchool['reading_score'].mean()
pct_math = pass_math_count_ser*100/tot_student_s
pct_read = pass_read_count_ser*100/tot_student_s
pct_overall = (pct_math + pct_read)/2


# school_types = school_data.set_index(["school_name"])["type"]

# combine calculated metrics into a mega dataframe
school_perf = pd.DataFrame({"School Type":sch_type,"Total Students":tot_student_s,
                           "Total School Budget":tot_budget_s,"Per Student Budget":stu_budget,
                           "Average Math Score":avg_math_s,"Average Reading Score":avg_read_s,
                           "% Passing Math":pct_math,"% Passing Reading":pct_read,
                            "% Overall Passing Rate":pct_overall})

# format budget columns
# school_perf["Total School Budget"] = school_perf["Total School Budget"].map("${:.2f}".format)
# school_perf["Per Student Budget"] = school_perf["Per Student Budget"].map("${:.2f}".format)


In [111]:
# Sort School Performance based on highest Overall Passing (descending) & display first 5 rows
top_schools = school_perf.sort_values(["% Overall Passing Rate"], ascending=False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,[Charter],1858,[1081356],[582.0],83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,[Charter],1635,[1043130],[638.0],83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,[Charter],962,[585858],[609.0],83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,[Charter],1468,[917500],[625.0],83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,[Charter],2283,[1319574],[578.0],83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [56]:
# Sort School Performance based on lowest Overall Passing & display first 5 rows
bottom_schools = school_perf.sort_values(["% Overall Passing Rate"], ascending=True)
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,[District],3999,[2547363],[637.0],76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,[District],2949,[1884411],[639.0],76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,[District],2917,[1910635],[655.0],76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,[District],4761,[3094650],[650.0],77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,[District],2739,[1763916],[644.0],77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [57]:
# Create a reduced DataFrame with columns needed
scores_math = data_combined.loc[:,["school_name","grade","math_score"]]
scores_math = scores_math.set_index('school_name')

# Create series for each grade level
nine = scores_math.loc[scores_math["grade"] == "9th"]
ten = scores_math.loc[scores_math["grade"] == "10th"]
eleven = scores_math.loc[scores_math["grade"] == "11th"]
twelve = scores_math.loc[scores_math["grade"] == "12th"]

# Group each series by school name and take averages
grp_nine = nine.groupby(['school_name']).mean()
grp_ten = ten.groupby(['school_name']).mean()    
grp_eleven = eleven.groupby(['school_name']).mean()
grp_twelve = twelve.groupby(['school_name']).mean()

# create dataframes out of each series
grp_nine_df = pd.DataFrame(grp_nine)
grp_ten_df = pd.DataFrame(grp_ten)
grp_eleven_df = pd.DataFrame(grp_eleven)
grp_twelve_df = pd.DataFrame(grp_twelve)

# merge into one dataframe with all grades and rename columns
math_by_grd = pd.merge(grp_nine_df,grp_ten_df, on='school_name')
math_by_grd = math_by_grd.rename(columns = {'math_score_x':'9th','math_score_y':'10th'})
math_by_grd = pd.merge(math_by_grd,grp_eleven_df, on='school_name')
math_by_grd = math_by_grd.rename(columns = {'math_score':'11th'})
math_by_grd = pd.merge(math_by_grd,grp_twelve_df, on='school_name')
math_by_grd = math_by_grd.rename(columns = {'math_score':'12th'})

math_by_grd

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [58]:
# Create a reduced DataFrame with columns needed
scores_read = data_combined.loc[:,["school_name","grade","reading_score"]]
scores_read = scores_read.set_index('school_name')

# Create series for each grade level
nine_r = scores_read.loc[scores_read["grade"] == "9th"]
ten_r = scores_read.loc[scores_read["grade"] == "10th"]
eleven_r = scores_read.loc[scores_read["grade"] == "11th"]
twelve_r = scores_read.loc[scores_read["grade"] == "12th"]

# Group each series by school name and take averages
grp_9r = nine_r.groupby(['school_name']).mean()
grp_10r = ten_r.groupby(['school_name']).mean()    
grp_11r = eleven_r.groupby(['school_name']).mean()
grp_12r = twelve_r.groupby(['school_name']).mean()

# create dataframes out of each series
grp_9r_df = pd.DataFrame(grp_9r)
grp_10r_df = pd.DataFrame(grp_10r)
grp_11r_df = pd.DataFrame(grp_11r)
grp_12r_df = pd.DataFrame(grp_12r)

# merge into one dataframe with all grades and rename columns
read_by_grd = pd.merge(grp_9r_df,grp_10r_df, on='school_name')
read_by_grd = read_by_grd.rename(columns = {'reading_score_x':'9th','reading_score_y':'10th'})
read_by_grd = pd.merge(read_by_grd,grp_11r_df, on='school_name')
read_by_grd = read_by_grd.rename(columns = {'reading_score':'11th'})
read_by_grd = pd.merge(read_by_grd,grp_12r_df, on='school_name')
read_by_grd = read_by_grd.rename(columns = {'reading_score':'12th'})

read_by_grd

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [97]:
# Set up bins for grouping spending
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

# Create reduced dataframe from school performance with needed columns
school_spend = pd.DataFrame(school_perf[['Per Student Budget', 'Average Math Score', 'Average Reading Score',
       '% Passing Math', '% Passing Reading', '% Overall Passing Rate']])

school_spend.reset_index(drop=True, inplace=True)

school_spend['Per Student Budget'] = school_spend['Per Student Budget'].astype('float')


In [98]:
school_spend["Spending Ranges (per student)"] = pd.cut(school_spend["Per Student Budget"], 
                                                       spending_bins, labels=group_names)

school_spend = school_spend.groupby("Spending Ranges (per student)")

school_spend_grp.head()

,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate,Spending Ranges (per student)
0,628.0,77.048432,81.033963,66.680064,81.933280,74.306672,$615-645
1,582.0,83.061895,83.975780,94.133477,97.039828,95.586652,<$585
2,639.0,76.711767,81.158020,65.988471,80.739234,73.363852,$615-645
3,644.0,77.102592,80.746258,68.309602,79.299014,73.804308,$615-645
4,625.0,83.351499,83.816757,93.392371,97.138965,95.265668,$615-645
5,652.0,77.289752,80.934412,66.752967,80.862999,73.807983,$645-675
6,581.0,83.803279,83.814988,92.505855,96.252927,94.379391,<$585
7,655.0,76.629414,81.182722,65.683922,81.316421,73.500171,$645-675
8,650.0,77.072464,80.966394,66.057551,81.222432,73.639992,$645-675
9,609.0,83.839917,84.044699,94.594595,95.945946,95.270270,$585-615


## Scores by School Size

* Perform the same operations as above, based on school size.

In [104]:
# Set up bins and group names
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Create reduced dataframe from school performance with needed columns
school_size = pd.DataFrame(school_perf[['Total Students', 'Average Math Score', 'Average Reading Score',
       '% Passing Math', '% Passing Reading', '% Overall Passing Rate']])

school_size.reset_index(drop=True, inplace=True)
school_size

,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,4976,77.048432,81.033963,66.680064,81.933280,74.306672
1,1858,83.061895,83.975780,94.133477,97.039828,95.586652
2,2949,76.711767,81.158020,65.988471,80.739234,73.363852
3,2739,77.102592,80.746258,68.309602,79.299014,73.804308
4,1468,83.351499,83.816757,93.392371,97.138965,95.265668
5,4635,77.289752,80.934412,66.752967,80.862999,73.807983
6,427,83.803279,83.814988,92.505855,96.252927,94.379391
7,2917,76.629414,81.182722,65.683922,81.316421,73.500171
8,4761,77.072464,80.966394,66.057551,81.222432,73.639992
9,962,83.839917,84.044699,94.594595,95.945946,95.270270


In [108]:
school_size["School Size"] = pd.cut(school_size["Total Students"],size_bins, labels=group_names)

school_size_grp = school_size.groupby("School Size")

school_size_grp.head()

ValueError: setting an array element with a sequence

## Scores by School Type

* Perform the same operations as above, based on school type.